In [ ]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296) 
  
# data (as pandas dataframes) 
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets 
  
# metadata 
print(diabetes_130_us_hospitals_for_years_1999_2008.metadata) 
# variable information

In [ ]:
display(diabetes_130_us_hospitals_for_years_1999_2008.keys())
a = diabetes_130_us_hospitals_for_years_1999_2008['data']
(X['admission_type_id']== 2).sum()
df =a['features']
a['ids']
#df = pd.concat([a['ids'],a['features']],axis = 1)

data = df.copy()
display(data)

In [ ]:
def split_train_test_considering_patient_nbr(data):
    from ucimlrepo import fetch_ucirepo
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split

    diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)
    #id_patients = diabetes_130_us_hospitals_for_years_1999_2008.data.ids
    #full_dataset = diabetes_130_us_hospitals_for_years_1999_2008.data.original
    id_patients = data['ids']
    full_dataset = data['original']

    # Identify unique patient numbers
    unique_patient_nbrs = id_patients['patient_nbr'].unique()
    # Split patient numbers into train and test sets
    train_patient_nbrs, test_patient_nbrs = train_test_split(unique_patient_nbrs, test_size=0.1, random_state=42)

    # Filter rows based on train and test patient numbers
    train_set = full_dataset[full_dataset['patient_nbr'].isin(train_patient_nbrs)]
    test_set = full_dataset[full_dataset['patient_nbr'].isin(test_patient_nbrs)]

    # Check that there's no id in the train set that is also in the test set:
    list_train_patients_id = list(train_set["patient_nbr"])
    uniques_list_train = set(list_train_patients_id)
    list_test_patients_id = list(test_set["patient_nbr"])
    uniques_list_test = set(list_test_patients_id)
    for i in list(uniques_list_train):
        if i in list(uniques_list_test):
            print("problem - id both in train and test")

    return train_set,test_set

dataset = [train_set,test_set] = split_train_test_considering_patient_nbr(a)


# check statistics of labels in training and test:
train_readmitted_stat = dataset[0]["readmitted"].value_counts() / len(dataset[0])
print("training dataset readmitted statistics", train_readmitted_stat)
test_readmitted_stat =dataset[1]["readmitted"].value_counts() / len(dataset[1])
print("test dataset readmitted statistics", test_readmitted_stat)
# Check statistics of gender in training and test:
train_gender_stat = dataset[0]["gender"].value_counts() / len(dataset[0])
print("training dataset gender statistics", train_gender_stat)
test_gender_stat =dataset[1]["gender"].value_counts() / len(dataset[1])
print("test dataset gender statistics", test_gender_stat)
train_set

In [ ]:
# Filtering data: For the subplots of 'diag_1', 'diag_2', 'diag_3', and 'medical_specialty', ,'num_lab_procedures', 'num_medications' where the number of distinct values is between 848-954,
# reading the x-labels and visually differentiating between columns becomes challenging. In these subplots, we will visualize only the top 20 common diagnoses.
columns_of_interest = ['diag_1', 'diag_2', 'diag_3', 'medical_specialty','num_lab_procedures', 'num_medications']
# Get the top 20 categories for each column
top_20_categories = {}
for column in columns_of_interest:
    top_20_categories[column] = train_set[column].value_counts().nlargest(20).index
# Filter the DataFrame to include only the rows with the top 20 categories for each column
filtered_data = train_set[
    (train_set['diag_1'].isin(top_20_categories['diag_1'])) &
    (train_set['diag_2'].isin(top_20_categories['diag_2'])) &
    (train_set['diag_3'].isin(top_20_categories['diag_3'])) &
    (train_set['medical_specialty'].isin(top_20_categories['medical_specialty'])) &
    (train_set['num_lab_procedures'].isin(top_20_categories['num_lab_procedures'])) &
    (train_set['num_medications'].isin(top_20_categories['num_medications']))]


In [ ]:
# creating a list of the numerical and categorical features
numerical = list(train_set.select_dtypes(include=['int64', 'float64']).drop(columns=['discharge_disposition_id','admission_type_id','admission_source_id', 'encounter_id', 'patient_nbr']).columns)
categorical = list(train_set.select_dtypes(include=['object']).drop(columns= ['payer_code']).columns)
rcol_names_list = filtered_data.columns.tolist()
remove_list = ['admission_type_id','discharge_disposition_id','encounter_id','admission_type_id','admission_source_id','payer_code','patient_nbr']
col_names_list = [i for i in rcol_names_list if i not in remove_list]

In [ ]:
# Function for plotting our data
def plotting(plot_type, C_list, C_data, the_label=None):
    lenl = len(C_list)//3
    fig, ax = plt.subplots(lenl, 3, figsize=(20,5*lenl))
    for variable, subplot in zip(C_list, ax.flatten()):
        if the_label == None:
            # function for plotting counts of multiple variables
            plot_type(x=C_data[variable], ax=subplot, color='#86bf91', zorder=2, width=0.9)
            # customize plot desighn
            subplot.set_ylabel("Count", labelpad=8, size=12)
            subplot.set_xlabel(variable, labelpad=8, size=12)
            # get the total count of the type column
            total = C_data[variable].count()
            subplot.bar_label(subplot.containers[0], fmt=lambda x: f'{(x/total)*100:0.1f}%')
            # add space at the end of the bar for the labels
            subplot.margins(x=0.01)

        else:
            if plot_type == sns.heatmap:
                heatmap_data = pd.crosstab(index=C_data[variable], columns=C_data[the_label])
                plot_type(data=heatmap_data, cmap="crest")

            else:
                # plotting multiple variables against one variable
                plot_type(x=C_data[the_label], y=C_data[variable], ax=subplot, color='#86bf91', zorder=2, width=0.9)

            # customize plot desighn
            subplot.set_ylabel(variable, labelpad=8, size=12)
            subplot.set_xlabel(the_label, labelpad=8, size=12)
        subplot.spines['right'].set_visible(False)
        subplot.spines['top'].set_visible(False)

        if C_list==categorical:
            # Rotating x-axis labels for better readability
            for label in subplot.get_xticklabels():
                label.set_rotation(90)

    plt.tight_layout()

# Initial analysis
### Let's look at the dataset

In [ ]:
train_set.head()

### Univariate Analysis (visualizing single variables)
### We atart with looking at the attributes' datatypes and number of missing values.

In [ ]:
train_set.info()

descriptive statistics of the data

In [ ]:
train_set.describe()

# Assessment of Missing Values
### Per Column and Overall Analysis

In [ ]:
# Check for missing values in each column
missing_values = train_set.isnull().sum()

# Display columns with missing values (if any)
columns_with_missing_values = missing_values[missing_values > 0]
if not columns_with_missing_values.empty:
    print("Columns with missing values:")
    print(columns_with_missing_values)
else:
    print("No missing values found in any column.")

full_data_nan = train_set.isna().sum().sum()
data_shape = train_set.shape[0]*train_set.shape[1]
full_nans_perc = full_data_nan/data_shape*100

print('Missing values percentage in the dataset:',full_nans_perc,'%')

### Histogram Displaying Percentage of Missing Values Across Features

In [ ]:
# Adjusting the 'none' value in two columns to 'not measured' - it is not a missing value by meaning
train_set['max_glu_serum'] = train_set['max_glu_serum'].fillna('not measured')
train_set['A1Cresult'] = train_set['A1Cresult'].fillna('not measured')
train_set['readmitted'] = train_set['readmitted'].fillna('NO')
train_set['max_glu_serum']

# Calculate the percentage of missing values per column
nan_percentages = train_set.isnull().mean() * 100

# Filter columns with missing values and their percentages
filtered_columns = nan_percentages[nan_percentages > 0].index
nan_percentage = nan_percentages[nan_percentages > 0].values

nans = sns.barplot(x=filtered_columns , y=nan_percentage, color='#86bf91', zorder=2, width=0.9)
# customize plot design
nans.set_ylabel("Missing values percentage (%)", labelpad=8, size=12)
nans.spines['right'].set_visible(False)
nans.spines['top'].set_visible(False)

for index, value in enumerate(nan_percentage):
    nans.text(index, value + 0.5, f'{value:.1f}%', ha='center', color='black')

# Rotating x-axis labels for better readability
for label in nans.get_xticklabels():
    label.set_rotation(90)

# Analyzing categorical variables
### Categorical variables distribution visualized using histogram and to visualize unique values and outliers

In [ ]:
plotting(sns.countplot, categorical, filtered_data)

# Analyzing numerical variables
### Plotting numerical feature counts using histogram

In [ ]:
plotting(sns.countplot, numerical, filtered_data)

# Analyzing relationship between variables
### Plotting heatmap for pearson correlation between numerical variables indluding the label (readmitted)

In [ ]:
train_set['readmitted'] = train_set['readmitted'].apply(lambda x : 1 if x == '>30'
                                                            else ( 0 if x == '<30'
                                                            else ( None)))
numerical.append('readmitted')
cor_df_num = train_set[numerical].corr(method = "pearson").round(2)
sns.heatmap(cor_df_num, vmin=0, vmax=1, annot=True, cmap="crest")

### Plotting the intersection of readmission status with categorical features to uncover correlations between these features and the label

In [ ]:
# Plot the readmittion rate for of the relevant drugs dist:
train_set.loc[train_set['readmitted']=='>30','readmitted']='other'
train_set.loc[train_set['readmitted']=='NO','readmitted']='other'

lenl = len(categorical)//3
fig, ax = plt.subplots(lenl, 3, figsize=(20,5*lenl))
for variable, subplot in zip(categorical, ax.flatten()):
    sns.histplot(ax=subplot ,data=train_set, x=train_set[variable], hue="readmitted", stat='density', common_norm=False, binwidth=1,
                 kde=False, kde_kws={'bw_adjust': 5}, color='#86bf91')
plt.show()


In [ ]:
# another visualization for categorical features and the label: 3 features example
exmp_cat = ['age', 'medical_specialty', 'insulin']
for cat in exmp_cat:
    # Assuming df is your DataFrame
    set_age = filtered_data[cat]
    set_age.value_counts()

    # Group by colA and then apply value_counts on colB within each group
    result = filtered_data.groupby(cat)['readmitted'].value_counts(normalize=False)

    # Sort a specific result from colB inside the result variable
    sorted_result = result[:, '<30'].sort_values(ascending=False)
    sorted_result

    # Get the value counts of colA
    value_counts_colA = set_age.value_counts()

    # Calculate the length of each relevant value count of colA and divide the sorted_result by it
    sorted_result_normalized = sorted_result / value_counts_colA
    sorted_result_normalized

    # Assuming sorted_result_normalized is the Series containing the normalized rates

    # Plot the data with Seaborn
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=sorted_result_normalized.index, y=sorted_result_normalized.values, marker='o',color='#86bf91')

    # Add labels and title

    plt.ylabel('Normalized readmission rate within 30 days')
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    # Add dotted grid lines
    plt.grid(True, linestyle='--')
    # Rotating x-axis labels for better readability

    for label in plt.gca().get_xticklabels():
        label.set_rotation(90)

    # Show the plot
    plt.show()



# Data Quality and cleanliness:
### Plotting cumulative Maximum Occurrence and Percentage per Column for Outlier Detection Impact

In [ ]:
# Function to find category with max occurrence and its percentage for each column
def find_max_occurrence_and_percentage(column):
    value_counts = column.value_counts()
    max_category = value_counts.idxmax()
    max_count = value_counts.max()
    total_count = column.count()
    percentage = (max_count / total_count) * 100
    return max_category, percentage

# Iterate over columns and find category with max occurrence and percentage
result = {}
for column in categorical:
    max_category, percentage = find_max_occurrence_and_percentage(train_set[column])
    result[column] = percentage

#Comulative Histogram - The Pecentage of Occoarance in the Most Common Catagory
h = sns.histplot(result.values(),binwidth=5,binrange = [0,100],cumulative = True,legend=False, color='#86bf91', zorder=2)
plt.vlines(x=95, ymin=0, ymax=40, colors='r', linestyles='dashed')
plt.xlabel('Pecentage of occurrence in the most common catagory [%]')
plt.title('Comulative Histogram - The Pecentage of occurrence in the Most Common Catagory')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)


#filter by threshold
filtered_keys = [key for key, value in result.items() if value < 95]

# 'Histogram of Missing Values Percaentage'
plt.figure()
n_subjects = len(df.index)
df_new = pd.Series(df.isna().sum()).reset_index(drop = True)/n_subjects*100
sns.histplot(df_new,binwidth= 5,cumulative = False,legend=False,binrange = [0,100],color='#86bf91', zorder=2)
plt.xlabel('Missing values [%]')
plt.title('Histogram of Missing Values Percaentage')
plt.vlines(x=30, ymin=0, ymax=47, colors='r', linestyles='dashed')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

### Quantifying Data Loss per Outlier Quantile

In [ ]:
# Define a function to remove outliers using the IQR method for a specific column
def remove_outliers_iqr(df, column, q=0.10):
    # Calculate the 10th and 90th percentiles
    lower_bound = df[column].quantile(q)
    upper_bound = df[column].quantile(1-q)
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Plot the percentage of data lost for each quantile to set a right outliers drop for our data

quantiles = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30]  # Quantiles to calculate
percent_lost_dict = {}

# Calculate the percentage of numerical data lost for different quantiles
for q in quantiles:
    temp_data = train_set[numerical].copy()  # Make a copy of the original data
    for column in numerical:
        outliers = remove_outliers_iqr(temp_data, column, q)
        temp_data.drop(outliers.index, inplace=True)
    percent_lost_dict[q] = ((len(train_set[numerical]) - len(temp_data)) / len(train_set[numerical])) * 100

# Plot the percentage of data lost for each quantile
ax = sns.lineplot(x=list(percent_lost_dict.keys()), y=list(percent_lost_dict.values()), marker='o', color='#86bf91', zorder=2)
plt.title('Percentage of numerical Data Lost for Different Quantiles')
plt.xlabel('Quantile (%)')
plt.ylabel('Percentage of Data Lost (%)')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim(left=0)  # Set the lower limit of x-axis to 0

plt.show()